In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName("IPL data analysis").getOrCreate()

# Data Preparation

In [3]:
df = spark.read.csv('IPL Ball-by-Ball 2008-2020.csv',inferSchema=True,header=True)

In [4]:
df.show(1)

+------+------+----+----+----------+-----------+---------+------------+----------+----------+------------+---------+--------------+----------------+-------+-----------+--------------------+--------------------+
|    id|inning|over|ball|   batsman|non_striker|   bowler|batsman_runs|extra_runs|total_runs|non_boundary|is_wicket|dismissal_kind|player_dismissed|fielder|extras_type|        batting_team|        bowling_team|
+------+------+----+----+----------+-----------+---------+------------+----------+----------+------------+---------+--------------+----------------+-------+-----------+--------------------+--------------------+
|335982|     1|   6|   5|RT Ponting|BB McCullum|AA Noffke|           1|         0|         1|           0|        0|            NA|              NA|     NA|         NA|Kolkata Knight Ri...|Royal Challengers...|
+------+------+----+----+----------+-----------+---------+------------+----------+----------+------------+---------+--------------+----------------+-------+

In [5]:
len(df.columns)

18

In [6]:
print(df.count(),",",len(df.columns))

193468 , 18


In [7]:
df1 = spark.read.csv('IPL Matches 2008-2020.csv',inferSchema=True,header=True)

In [8]:
df1.show() 

+------+----------+----------+---------------+--------------------+-------------+--------------------+--------------------+--------------------+-------------+--------------------+-------+-------------+----------+------+-----------+--------------+
|    id|      city|      date|player_of_match|               venue|neutral_venue|               team1|               team2|         toss_winner|toss_decision|              winner| result|result_margin|eliminator|method|    umpire1|       umpire2|
+------+----------+----------+---------------+--------------------+-------------+--------------------+--------------------+--------------------+-------------+--------------------+-------+-------------+----------+------+-----------+--------------+
|335982| Bangalore|2008-04-18|    BB McCullum|M Chinnaswamy Sta...|            0|Royal Challengers...|Kolkata Knight Ri...|Royal Challengers...|        field|Kolkata Knight Ri...|   runs|          140|         N|    NA|  Asad Rauf|   RE Koertzen|
|335983|Chan

In [9]:
df1.count()

816

In [10]:
data=df.join(df1,on='id',how='left')

In [11]:
data.show(2)

+------+------+----+----+-----------+-----------+---------+------------+----------+----------+------------+---------+--------------+----------------+-------+-----------+--------------------+--------------------+---------+----------+---------------+--------------------+-------------+--------------------+--------------------+--------------------+-------------+--------------------+------+-------------+----------+------+---------+-----------+
|    id|inning|over|ball|    batsman|non_striker|   bowler|batsman_runs|extra_runs|total_runs|non_boundary|is_wicket|dismissal_kind|player_dismissed|fielder|extras_type|        batting_team|        bowling_team|     city|      date|player_of_match|               venue|neutral_venue|               team1|               team2|         toss_winner|toss_decision|              winner|result|result_margin|eliminator|method|  umpire1|    umpire2|
+------+------+----+----+-----------+-----------+---------+------------+----------+----------+------------+-------

In [12]:
data.head(2)[0]

Row(id=335982, inning=1, over=6, ball=5, batsman='RT Ponting', non_striker='BB McCullum', bowler='AA Noffke', batsman_runs=1, extra_runs=0, total_runs=1, non_boundary=0, is_wicket=0, dismissal_kind='NA', player_dismissed='NA', fielder='NA', extras_type='NA', batting_team='Kolkata Knight Riders', bowling_team='Royal Challengers Bangalore', city='Bangalore', date='2008-04-18', player_of_match='BB McCullum', venue='M Chinnaswamy Stadium', neutral_venue=0, team1='Royal Challengers Bangalore', team2='Kolkata Knight Riders', toss_winner='Royal Challengers Bangalore', toss_decision='field', winner='Kolkata Knight Riders', result='runs', result_margin='140', eliminator='N', method='NA', umpire1='Asad Rauf', umpire2='RE Koertzen')

In [13]:
from pyspark.sql import functions as F 

# 1. Most number of runs scored by a batsman

In [14]:
data.columns

['id',
 'inning',
 'over',
 'ball',
 'batsman',
 'non_striker',
 'bowler',
 'batsman_runs',
 'extra_runs',
 'total_runs',
 'non_boundary',
 'is_wicket',
 'dismissal_kind',
 'player_dismissed',
 'fielder',
 'extras_type',
 'batting_team',
 'bowling_team',
 'city',
 'date',
 'player_of_match',
 'venue',
 'neutral_venue',
 'team1',
 'team2',
 'toss_winner',
 'toss_decision',
 'winner',
 'result',
 'result_margin',
 'eliminator',
 'method',
 'umpire1',
 'umpire2']

In [15]:
df2=data.groupBy('batsman').agg(F.sum('batsman_runs').alias('most_runs'))

In [16]:
most_runs=df2.sort(F.col("most_runs").desc())

In [17]:
most_runs.show(10) 

+--------------+---------+
|       batsman|most_runs|
+--------------+---------+
|       V Kohli|     5878|
|      SK Raina|     5368|
|     DA Warner|     5254|
|     RG Sharma|     5230|
|      S Dhawan|     5197|
|AB de Villiers|     4849|
|      CH Gayle|     4772|
|      MS Dhoni|     4632|
|    RV Uthappa|     4607|
|     G Gambhir|     4217|
+--------------+---------+
only showing top 10 rows



# 2. Most number of wickets taken by a bowler

In [18]:
for i in [data[0] for data in data.select(F.col('dismissal_kind')).distinct().collect()]:
    data.filter(F.col('dismissal_kind')==i).agg(F.count('dismissal_kind')).show() 

+---------------------+
|count(dismissal_kind)|
+---------------------+
|                  294|
+---------------------+

+---------------------+
|count(dismissal_kind)|
+---------------------+
|                   12|
+---------------------+

+---------------------+
|count(dismissal_kind)|
+---------------------+
|               183973|
+---------------------+

+---------------------+
|count(dismissal_kind)|
+---------------------+
|                 1700|
+---------------------+

+---------------------+
|count(dismissal_kind)|
+---------------------+
|                  571|
+---------------------+

+---------------------+
|count(dismissal_kind)|
+---------------------+
|                    2|
+---------------------+

+---------------------+
|count(dismissal_kind)|
+---------------------+
|                  269|
+---------------------+

+---------------------+
|count(dismissal_kind)|
+---------------------+
|                   11|
+---------------------+

+---------------------+
|count(d

In [19]:
###like value_counts in pandas we can do the same with the help of following code in pyspark
data.groupBy(F.col('dismissal_kind')).count().orderBy('count').sort(F.col("count").desc()) .show()

+--------------------+------+
|      dismissal_kind| count|
+--------------------+------+
|                  NA|183973|
|              caught|  5743|
|              bowled|  1700|
|             run out|   893|
|                 lbw|   571|
|             stumped|   294|
|   caught and bowled|   269|
|          hit wicket|    12|
|        retired hurt|    11|
|obstructing the f...|     2|
+--------------------+------+



In [20]:
values=['NA','run out','hit wicket','retired hurt','obstructing the field']

In [21]:
data.columns

['id',
 'inning',
 'over',
 'ball',
 'batsman',
 'non_striker',
 'bowler',
 'batsman_runs',
 'extra_runs',
 'total_runs',
 'non_boundary',
 'is_wicket',
 'dismissal_kind',
 'player_dismissed',
 'fielder',
 'extras_type',
 'batting_team',
 'bowling_team',
 'city',
 'date',
 'player_of_match',
 'venue',
 'neutral_venue',
 'team1',
 'team2',
 'toss_winner',
 'toss_decision',
 'winner',
 'result',
 'result_margin',
 'eliminator',
 'method',
 'umpire1',
 'umpire2']

In [22]:
most_wickets = data.filter(~F.col('dismissal_kind').isin(values)).groupBy('bowler') \
    .agg(F.count('is_wicket')).sort(F.col('count(is_wicket)').desc()).withColumnRenamed('count(is_wicket)', 'most_wickets')

In [23]:
most_wickets.show(5) 

+---------------+------------+
|         bowler|most_wickets|
+---------------+------------+
|     SL Malinga|         170|
|       A Mishra|         160|
|      PP Chawla|         156|
|       DJ Bravo|         153|
|Harbhajan Singh|         149|
+---------------+------------+
only showing top 5 rows



# 3. Most matches win by a team

In [24]:
data.head()

Row(id=335982, inning=1, over=6, ball=5, batsman='RT Ponting', non_striker='BB McCullum', bowler='AA Noffke', batsman_runs=1, extra_runs=0, total_runs=1, non_boundary=0, is_wicket=0, dismissal_kind='NA', player_dismissed='NA', fielder='NA', extras_type='NA', batting_team='Kolkata Knight Riders', bowling_team='Royal Challengers Bangalore', city='Bangalore', date='2008-04-18', player_of_match='BB McCullum', venue='M Chinnaswamy Stadium', neutral_venue=0, team1='Royal Challengers Bangalore', team2='Kolkata Knight Riders', toss_winner='Royal Challengers Bangalore', toss_decision='field', winner='Kolkata Knight Riders', result='runs', result_margin='140', eliminator='N', method='NA', umpire1='Asad Rauf', umpire2='RE Koertzen')

In [25]:
df1.head()

Row(id=335982, city='Bangalore', date='2008-04-18', player_of_match='BB McCullum', venue='M Chinnaswamy Stadium', neutral_venue=0, team1='Royal Challengers Bangalore', team2='Kolkata Knight Riders', toss_winner='Royal Challengers Bangalore', toss_decision='field', winner='Kolkata Knight Riders', result='runs', result_margin='140', eliminator='N', method='NA', umpire1='Asad Rauf', umpire2='RE Koertzen')

In [26]:
most_matches_won=df1.groupBy('winner').count().sort(F.col('count').desc()).withColumnRenamed('count','most_matches_won')

In [27]:
most_matches_won.show(5) 

+--------------------+----------------+
|              winner|most_matches_won|
+--------------------+----------------+
|      Mumbai Indians|             120|
| Chennai Super Kings|             106|
|Kolkata Knight Ri...|              99|
|Royal Challengers...|              91|
|     Kings XI Punjab|              88|
+--------------------+----------------+
only showing top 5 rows



# 4. Most catches by a player

In [28]:
data.columns

['id',
 'inning',
 'over',
 'ball',
 'batsman',
 'non_striker',
 'bowler',
 'batsman_runs',
 'extra_runs',
 'total_runs',
 'non_boundary',
 'is_wicket',
 'dismissal_kind',
 'player_dismissed',
 'fielder',
 'extras_type',
 'batting_team',
 'bowling_team',
 'city',
 'date',
 'player_of_match',
 'venue',
 'neutral_venue',
 'team1',
 'team2',
 'toss_winner',
 'toss_decision',
 'winner',
 'result',
 'result_margin',
 'eliminator',
 'method',
 'umpire1',
 'umpire2']

In [29]:
data.head(2)

[Row(id=335982, inning=1, over=6, ball=5, batsman='RT Ponting', non_striker='BB McCullum', bowler='AA Noffke', batsman_runs=1, extra_runs=0, total_runs=1, non_boundary=0, is_wicket=0, dismissal_kind='NA', player_dismissed='NA', fielder='NA', extras_type='NA', batting_team='Kolkata Knight Riders', bowling_team='Royal Challengers Bangalore', city='Bangalore', date='2008-04-18', player_of_match='BB McCullum', venue='M Chinnaswamy Stadium', neutral_venue=0, team1='Royal Challengers Bangalore', team2='Kolkata Knight Riders', toss_winner='Royal Challengers Bangalore', toss_decision='field', winner='Kolkata Knight Riders', result='runs', result_margin='140', eliminator='N', method='NA', umpire1='Asad Rauf', umpire2='RE Koertzen'),
 Row(id=335982, inning=1, over=6, ball=6, batsman='BB McCullum', non_striker='RT Ponting', bowler='AA Noffke', batsman_runs=1, extra_runs=0, total_runs=1, non_boundary=0, is_wicket=0, dismissal_kind='NA', player_dismissed='NA', fielder='NA', extras_type='NA', battin

In [30]:
most_catches=data.filter(F.col('dismissal_kind') == 'caught').groupBy('fielder').agg(F.count('fielder')).sort(F.col('count(fielder)')
                                                                                                 .desc()).withColumnRenamed('count(fielder)', 'most_catches')

In [31]:
most_catches.show(5)

+--------------+------------+
|       fielder|most_catches|
+--------------+------------+
|    KD Karthik|         118|
|      MS Dhoni|         113|
|AB de Villiers|         103|
|      SK Raina|          99|
|     RG Sharma|          88|
+--------------+------------+
only showing top 5 rows



# 5. Most man of the match

In [32]:
df1.head(2)

[Row(id=335982, city='Bangalore', date='2008-04-18', player_of_match='BB McCullum', venue='M Chinnaswamy Stadium', neutral_venue=0, team1='Royal Challengers Bangalore', team2='Kolkata Knight Riders', toss_winner='Royal Challengers Bangalore', toss_decision='field', winner='Kolkata Knight Riders', result='runs', result_margin='140', eliminator='N', method='NA', umpire1='Asad Rauf', umpire2='RE Koertzen'),
 Row(id=335983, city='Chandigarh', date='2008-04-19', player_of_match='MEK Hussey', venue='Punjab Cricket Association Stadium, Mohali', neutral_venue=0, team1='Kings XI Punjab', team2='Chennai Super Kings', toss_winner='Chennai Super Kings', toss_decision='bat', winner='Chennai Super Kings', result='runs', result_margin='33', eliminator='N', method='NA', umpire1='MR Benson', umpire2='SL Shastri')]

In [33]:
most_MOM=df1.groupBy('player_of_match').count().sort(F.col('count').desc()).withColumnRenamed('count','most_MOM')

In [34]:
most_MOM.show(5)

+---------------+--------+
|player_of_match|most_MOM|
+---------------+--------+
| AB de Villiers|      23|
|       CH Gayle|      22|
|      RG Sharma|      18|
|      DA Warner|      17|
|       MS Dhoni|      17|
+---------------+--------+
only showing top 5 rows



# 6. Most sixes Hit by a player

In [35]:
data=data.withColumn('six_flag', F.when(F.col('batsman_runs') == F.lit(6), F.lit(1)
                                   ).otherwise(F.lit(0))) 

In [36]:
data.groupBy('six_flag').count().show()

+--------+------+
|six_flag| count|
+--------+------+
|       1|  8902|
|       0|184566|
+--------+------+



Method:1

In [37]:
data.groupBy('batsman').sum('six_flag').sort(F.col('sum(six_flag)').desc()).show(5)

+--------------+-------------+
|       batsman|sum(six_flag)|
+--------------+-------------+
|      CH Gayle|          349|
|AB de Villiers|          235|
|      MS Dhoni|          216|
|     RG Sharma|          214|
|       V Kohli|          202|
+--------------+-------------+
only showing top 5 rows



Method:2

In [38]:
most_sixes=data.groupBy('batsman').agg(F.sum('six_flag')).sort(F.col('sum(six_flag)').desc()).withColumnRenamed('sum(six_flag)','most_sixes')

In [39]:
most_sixes.show(10)

+--------------+----------+
|       batsman|most_sixes|
+--------------+----------+
|      CH Gayle|       349|
|AB de Villiers|       235|
|      MS Dhoni|       216|
|     RG Sharma|       214|
|       V Kohli|       202|
|    KA Pollard|       198|
|     DA Warner|       195|
|      SK Raina|       194|
|     SR Watson|       190|
|    RV Uthappa|       163|
+--------------+----------+
only showing top 10 rows



# 7. Most Fours by a player

In [40]:
data=data.withColumn('four_flag', F.when(F.col('batsman_runs') == F.lit(4), F.lit(1)
                                   ).otherwise(F.lit(0))) 

In [41]:
most_fours=data.groupBy('batsman').agg(F.sum('four_flag')).sort(F.col('sum(four_flag)').desc())

In [42]:
most_fours.show(5)

+---------+--------------+
|  batsman|sum(four_flag)|
+---------+--------------+
| S Dhawan|           591|
|DA Warner|           510|
|  V Kohli|           504|
| SK Raina|           493|
|G Gambhir|           492|
+---------+--------------+
only showing top 5 rows



# 8. Highest Strike rate of batsman

In [43]:
balls = data.groupBy('batsman').agg(F.count(F.col('ball'))).sort(
    F.col('count(ball)').desc()).withColumnRenamed('count(ball)', 'balls')

In [44]:
balls.show(5) 

+---------+-----+
|  batsman|balls|
+---------+-----+
|  V Kohli| 4609|
| S Dhawan| 4208|
|RG Sharma| 4088|
| SK Raina| 4041|
|DA Warner| 3819|
+---------+-----+
only showing top 5 rows



In [45]:
most_runs.show(5)

+---------+---------+
|  batsman|most_runs|
+---------+---------+
|  V Kohli|     5878|
| SK Raina|     5368|
|DA Warner|     5254|
|RG Sharma|     5230|
| S Dhawan|     5197|
+---------+---------+
only showing top 5 rows



In [46]:
sr_df=most_runs.join(balls,on='batsman',how='left')

In [47]:
sr_df.show(5)

+-------------+---------+-----+
|      batsman|most_runs|balls|
+-------------+---------+-----+
|Kuldeep Yadav|       57|   76|
|   S Anirudha|      136|  121|
|   TM Dilshan|     1153| 1047|
|      J Botha|      409|  364|
|   KA Pollard|     3023| 2107|
+-------------+---------+-----+
only showing top 5 rows



In [48]:
sr_df = sr_df.withColumn('strike_rate', (F.col('most_runs') / F.col('balls'))*100).select("*", F.round(F.col('strike_rate'), 2)) \
    .drop('strike_rate').withColumnRenamed('round(strike_rate, 2)', 'strike_rate')

In [49]:
sr_df.filter(F.col('balls') > 100).sort(F.col('strike_rate').desc()).show(15)  

+--------------+---------+-----+-----------+
|       batsman|most_runs|balls|strike_rate|
+--------------+---------+-----+-----------+
|    AD Russell|     1517|  882|      172.0|
|     K Gowtham|      186|  113|      164.6|
|   BCJ Cutting|      238|  146|     163.01|
|      N Pooran|      521|  323|      161.3|
|     SP Narine|      892|  573|     155.67|
|        MM Ali|      309|  199|     155.28|
|     CH Morris|      551|  360|     153.06|
|     JC Archer|      195|  128|     152.34|
| CR Brathwaite|      181|  120|     150.83|
|  Bipul Sharma|      187|  124|     150.81|
|     HH Pandya|     1349|  897|     150.39|
|      V Sehwag|     2728| 1833|     148.83|
|    GJ Maxwell|     1505| 1013|     148.57|
|AB de Villiers|     4849| 3264|     148.56|
|       RR Pant|     2079| 1416|     146.82|
+--------------+---------+-----+-----------+
only showing top 15 rows



# 9. Most matches won by a team

In [50]:
most_wins=df1.groupBy('winner').agg(F.count(F.col('winner'))).sort(F.col('count(winner)').desc()) \
    .withColumnRenamed('count(winner)', 'most_wins')

In [51]:
most_wins.show(5) 

+--------------------+---------+
|              winner|most_wins|
+--------------------+---------+
|      Mumbai Indians|      120|
| Chennai Super Kings|      106|
|Kolkata Knight Ri...|       99|
|Royal Challengers...|       91|
|     Kings XI Punjab|       88|
+--------------------+---------+
only showing top 5 rows



# 10. Most no balls balled by a baller

In [52]:
df.head(1)

[Row(id=335982, inning=1, over=6, ball=5, batsman='RT Ponting', non_striker='BB McCullum', bowler='AA Noffke', batsman_runs=1, extra_runs=0, total_runs=1, non_boundary=0, is_wicket=0, dismissal_kind='NA', player_dismissed='NA', fielder='NA', extras_type='NA', batting_team='Kolkata Knight Riders', bowling_team='Royal Challengers Bangalore')]

In [62]:
most_noballs=df.filter(F.col('extras_type') == "noballs").groupBy('bowler').agg(F.count(F.col('extras_type'))). \
sort(F.col('count(extras_type)').desc()).withColumnRenamed('count(extras_type)','most_noballs')

In [63]:
most_noballs.show(5)

+-----------+------------+
|     bowler|most_noballs|
+-----------+------------+
|S Sreesanth|          23|
|  JJ Bumrah|          22|
|   A Mishra|          21|
|   I Sharma|          21|
| SL Malinga|          18|
+-----------+------------+
only showing top 5 rows



# 11. Most stumpings by a wicketkeeper

In [66]:
most_stumping=data.filter(F.col('dismissal_kind') == 'stumped').groupBy('fielder').agg(F.count(F.col('dismissal_kind'))) \
.sort(F.col('count(dismissal_kind)').desc()).withColumnRenamed('count(dismissal_kind)','most_stumping')

In [67]:
most_stumping.show(5)

+------------+-------------+
|     fielder|most_stumping|
+------------+-------------+
|    MS Dhoni|           39|
|  RV Uthappa|           32|
|  KD Karthik|           30|
|     WP Saha|           20|
|AC Gilchrist|           16|
+------------+-------------+
only showing top 5 rows



# 12. Batsman who got runout most number of times

In [68]:
most_runouts=data.filter(F.col('dismissal_kind') == 'run out').groupBy('batsman').agg(F.count(F.col('dismissal_kind'))) \
.sort(F.col('count(dismissal_kind)').desc()).withColumnRenamed('count(dismissal_kind)','most_runouts')

In [69]:
most_runouts.show(5)

+----------+------------+
|   batsman|most_runouts|
+----------+------------+
|  MS Dhoni|          22|
| RG Sharma|          20|
|KD Karthik|          17|
|   V Kohli|          15|
|  S Dhawan|          15|
+----------+------------+
only showing top 5 rows



# 13. Batsman who got bowled most number of times

In [70]:
most_bowled=data.filter(F.col('dismissal_kind') == 'bowled').groupBy('batsman').agg(F.count(F.col('dismissal_kind'))) \
.sort(F.col('count(dismissal_kind)').desc()).withColumnRenamed('count(dismissal_kind)','most_times_bowled')

In [71]:
most_bowled.show(5)

+---------+-----------------+
|  batsman|most_times_bowled|
+---------+-----------------+
|SR Watson|               35|
| S Dhawan|               31|
|  V Kohli|               30|
|G Gambhir|               27|
|DA Warner|               25|
+---------+-----------------+
only showing top 5 rows

